In [1]:
# === Expense Tracker v2.1 (Colab-ready) ===
# Functions, CSV load/save, validation, immediate append, aligned reports, simple menu

import csv
import os
from datetime import date, datetime

CSV_PATH = "expenses.csv"
FIELDNAMES = ["date", "category", "amount", "note"]


# ---------- Utilities ----------
def ensure_csv_header(csv_path: str) -> None:
    """Create CSV with header if it doesn't exist."""
    if not os.path.exists(csv_path):
        with open(csv_path, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
            writer.writeheader()


def load_expenses(csv_path: str):
    """Load all expenses from CSV into a list of dicts."""
    ensure_csv_header(csv_path)
    expenses = []
    with open(csv_path, mode="r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                amt = float(row.get("amount", ""))
            except (ValueError, TypeError):
                print("⚠️ Skipping a row with invalid amount:", row)
                continue
            expenses.append({
                "date": row.get("date", ""),
                "category": row.get("category", ""),
                "amount": amt,
                "note": row.get("note", ""),
            })
    return expenses


def append_expense(csv_path: str, expense: dict) -> None:
    """Append one expense dict to CSV."""
    ensure_csv_header(csv_path)
    with open(csv_path, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writerow({
            "date": expense["date"],
            "category": expense["category"],
            "amount": expense["amount"],
            "note": expense["note"],
        })


# ---------- Interactive input ----------
def prompt_date_or_quit() -> str | None:
    """Prompt for date; '' -> today, 'q' -> quit; validates YYYY-MM-DD."""
    while True:
        d = input("Date (YYYY-MM-DD, Enter=today, 'q'=quit): ").strip()
        if d.lower() == "q":
            return None
        if d == "":
            return str(date.today())
        try:
            datetime.strptime(d, "%Y-%m-%d")
            return d
        except ValueError:
            print("⚠️ Use format YYYY-MM-DD (e.g., 2025-09-02).")


def add_expense_interactive() -> dict | None:
    """Collect one expense from the user; returns dict or None if quit."""
    d = prompt_date_or_quit()
    if d is None:
        return None

    category = input("Category (e.g., coffee, rent): ").strip()
    if not category:
        print("⚠️ Category is required. Cancelling.")
        return None

    while True:
        amt_text = input("Amount (e.g., 5 or 5.50): ").strip()
        try:
            amt = float(amt_text)
            break
        except ValueError:
            print("⚠️ Amount must be a number. Try again.")

    note = input("Note (optional, Enter to skip): ").strip()

    return {"date": d, "category": category, "amount": amt, "note": note}


# ---------- Reporting ----------
def summarize_expenses(expenses: list[dict]):
    """Return (total, by_category:dict, by_month:dict)."""
    total = sum(e["amount"] for e in expenses)

    by_category = {}
    for e in expenses:
        cat, amt = e["category"], e["amount"]
        by_category[cat] = by_category.get(cat, 0) + amt

    by_month = {}
    for e in expenses:
        mon = e["date"][:7] if e["date"] else "unknown"
        amt = e["amount"]
        by_month[mon] = by_month.get(mon, 0) + amt

    return total, by_category, by_month


def print_summary(expenses: list[dict]) -> None:
    """Pretty-print totals and last 5 entries."""
    total, by_cat, by_month = summarize_expenses(expenses)

    print("\n========================")
    print("      EXPENSE SUMMARY   ")
    print("========================")
    print(f"Total: {total:.2f}\n")

    print("By Category:")
    for cat, amt in sorted(by_cat.items(), key=lambda x: -x[1]):  # high → low
        print(f"- {cat:<14} {amt:>10.2f}")

    print("\nBy Month:")
    for mon in sorted(by_month.keys()):
        print(f"- {mon:<14} {by_month[mon]:>10.2f}")

    if expenses:
        print("\nLast 5 entries:")
        for e in expenses[-5:]:
            print(f"{e['date']} | {e['category']:<10} | {e['amount']:>7.2f} | {e['note']}")
    else:
        print("\n(No expenses recorded.)")


# ---------- Main menu ----------
def main():
    print("📁 Using CSV:", CSV_PATH)
    expenses = load_expenses(CSV_PATH)
    print(f"📥 Loaded {len(expenses)} existing expenses.")

    while True:
        print("\nMenu")
        print("1. Add expense")
        print("2. View summary")
        print("q. Quit")
        choice = input("Choose an option: ").strip().lower()

        if choice == "1":
            exp = add_expense_interactive()
            if exp is None:
                # user cancelled or hit 'q' at date prompt
                continue
            expenses.append(exp)
            append_expense(CSV_PATH, exp)
            print(f"✅ Saved: {exp['category']} - {exp['amount']:.2f} on {exp['date']}")
        elif choice == "2":
            print_summary(expenses)
        elif choice == "q":
            print("👋 Goodbye!")
            break
        else:
            print("⚠️ Invalid choice. Try again.")


# Run the app
if __name__ == "__main__":
    main()


📁 Using CSV: expenses.csv
📥 Loaded 0 existing expenses.

Menu
1. Add expense
2. View summary
q. Quit
Choose an option: 2

      EXPENSE SUMMARY   
Total: 0.00

By Category:

By Month:

(No expenses recorded.)

Menu
1. Add expense
2. View summary
q. Quit
Choose an option: 1
Date (YYYY-MM-DD, Enter=today, 'q'=quit): 
Category (e.g., coffee, rent): c
Amount (e.g., 5 or 5.50): 4
Note (optional, Enter to skip): 
✅ Saved: c - 4.00 on 2025-09-07

Menu
1. Add expense
2. View summary
q. Quit
Choose an option: 2

      EXPENSE SUMMARY   
Total: 4.00

By Category:
- c                    4.00

By Month:
- 2025-09              4.00

Last 5 entries:
2025-09-07 | c          |    4.00 | 

Menu
1. Add expense
2. View summary
q. Quit
Choose an option: q
👋 Goodbye!
